In [2]:
import pandas as pd
import math
from collections import namedtuple
from PSSimPy.simulator import ABMSim

In [3]:
GameState = namedtuple("GameState", 
                        ["t",                      # current period
                            "balances",               # list of balances for each player
                            "borrowed_amounts",       # list of borrowed amounts for each player
                            "outstanding_obligations" # list of total obligations for each player
                        ])

In [8]:
class GameTreeSearch:

    def __init__(self, n_players: int=3):
        self.initialize_state()

    def initialize_state(self, n_players=3):
        """Create an initial game state with zero balances/borrows and no obligations."""
        self.state = GameState(t=0,
                        balances=[0]*n_players,
                        borrowed_amounts=[0]*n_players,
                        outstanding_obligations=[0]*n_players)

    @staticmethod
    def apply_actions(state, actions, delta, gamma):
        """
        Given a state and a list of actions (0=delay, 1=pay) for each player,
        return the new state and the immediate cost (for each player) from these actions.
        """
        n = len(state.balances)
        new_balances = list(state.balances)
        new_borrowed = list(state.borrowed_amounts)
        new_obligations = list(state.outstanding_obligations)  # might get updated to 0 if paid or carried over if delayed

        immediate_costs = [0.0]*n
        
        # For each player p
        for p in range(n):
            if actions[p] == 1:  # pay
                # If balance >= obligations, pay from balance
                needed = new_obligations[p]
                if new_balances[p] >= needed:
                    new_balances[p] -= needed
                    new_obligations[p] = 0
                else:
                    # must borrow the shortfall
                    shortfall = needed - new_balances[p]
                    new_balances[p] = 0
                    new_borrowed[p] += shortfall
                    new_obligations[p] = 0
                    # cost for borrowing
                    immediate_costs[p] += gamma * shortfall
                
                # Attempt to repay borrowed if there's leftover balance
                if new_balances[p] > 0 and new_borrowed[p] > 0:
                    repay = min(new_balances[p], new_borrowed[p])
                    new_borrowed[p] -= repay
                    new_balances[p] -= repay

            else:  # delay
                # cost for delaying
                delay_cost = delta * new_obligations[p]
                immediate_costs[p] += delay_cost
                # obligations remain the same, carried forward

        return (GameState(t=state.t, 
                        balances=new_balances, 
                        borrowed_amounts=new_borrowed, 
                        outstanding_obligations=new_obligations),
                immediate_costs)

    @staticmethod
    def evolve_to_next_period(state, lambda_prob=0.5):
        """
        Evolve the state from period t to t+1, possibly adding new obligations with probability lambda_prob.
        In a full game tree, you could branch for each random event. 
        Here, as a single branch example, let's assume exactly one new obligation occurs for each player with probability lambda_prob.
        That means expected number of new obligations is n*lambda_prob, but for demonstration, we’ll just treat it as deterministic or skip.
        
        In a real game tree approach, you'd create multiple successors for each random event combination. 
        """
        n = len(state.balances)
        new_obligations = list(state.outstanding_obligations)
        
        # For demonstration, let's do a naive approach: 
        # each player with probability lambda_prob gets an obligation of 1 more unit. 
        # We'll treat it as expected or just add a single unit for everyone half the time, etc.
        # In a real enumeration, you'd create multiple branches and attach probabilities.
        
        # Let's forcibly add 1 to each player's obligations with a 50% chance
        # (This is a single branch example - in a full tree you'd branch).
        import random
        if random.random() < lambda_prob:
            for p in range(n):
                new_obligations[p] += 1
        
        return GameState(t=state.t+1,
                        balances=state.balances,
                        borrowed_amounts=state.borrowed_amounts,
                        outstanding_obligations=new_obligations)

    @staticmethod
    def is_terminal(state, n_periods=3):
        """Check if we've reached the last period."""
        return state.t >= n_periods

    @staticmethod
    def game_tree_search(state, n_periods, delta, gamma, lambda_prob):
        """
        Recursively explore the game tree from the given state, returning 
        the cost (for each player) under optimal or equilibrium play (depending on solution concept).
        In this naive version, we'll just minimize the sum of expected costs for each player 
        (not fully game-theoretic). For a real multi-player game, you'd do 
        backward induction or iterative best response.
        """
        # If we are at a terminal state, cost is 0 from here on out.
        if GameTreeSearch.is_terminal(state, n_periods):
            return [0.0]*len(state.balances), []

        n = len(state.balances)
        
        # All combinations of actions (pay/delay) for n players: 2^n
        all_action_profiles = []
        for i in range(2**n):
            # decode i into a list of 0/1 actions
            actions = []
            tmp = i
            for _ in range(n):
                actions.append(tmp % 2)
                tmp //= 2
            actions.reverse()  # so player 0's action is at index 0, etc.
            all_action_profiles.append(actions)
        
        # For each action profile, apply immediate cost, then evolve to next state, 
        # then recursively find the future cost
        best_cost_sum = math.inf
        best_profile = None
        best_future_costs = None
        
        for profile in all_action_profiles:
            next_state, immediate_costs = GameTreeSearch.apply_actions(state, profile, delta, gamma)
            
            # Evolve to next period (for demonstration, ignoring that we might branch on random events)
            next_state = GameTreeSearch.evolve_to_next_period(next_state, lambda_prob)
            
            future_costs, _ = GameTreeSearch.game_tree_search(next_state, n_periods, delta, gamma, lambda_prob)
            
            # Total cost for each player is immediate + future
            total_costs = [ic + fc for ic, fc in zip(immediate_costs, future_costs)]
            sum_of_costs = sum(total_costs)
            
            # In a fully game-theoretic setting, you'd want to store the entire cost vector 
            # and do a proper multi-criteria selection (like Nash equilibrium checks). 
            # Here we just pick the profile that yields the minimal sum of costs.
            if sum_of_costs < best_cost_sum:
                best_cost_sum = sum_of_costs
                best_profile = profile
                best_future_costs = total_costs
        
        # Return the chosen cost vector (naive approach) and the chosen profile
        return best_future_costs, best_profile


In [16]:

# Example usage
n_players = 7
n_periods = 7
delta = 1.0   # delay cost
gamma = 0.2   # borrowing fee
lambda_prob = 0.5

GameSearch = GameTreeSearch(n_players)

# We'll run a naive game tree search from the initial state
# (in a real setting, random events should be branched out, and 
# multi-player optimal strategy might need more advanced solution concepts)
final_costs, first_profile = GameSearch.game_tree_search(GameSearch.state, n_periods, delta, gamma, lambda_prob)

print("Approximate minimal sum-of-costs solution from initial state:")
print(f"First action profile chosen: {['Pay' if a==1 else 'Delay' for a in first_profile]}")
print(f"Resulting total cost (per player) by the end of the game: {final_costs}")

Approximate minimal sum-of-costs solution from initial state:
First action profile chosen: ['Delay', 'Delay', 'Delay']
Resulting total cost (per player) by the end of the game: [0.0, 0.0, 0.0]
